In [ ]:
import speech_recognition as sr
#import pocketsphinx

In [ ]:
import os
import pyaudio
import urllib
import pyttsx3
from nltk.corpus import wordnet as wn

In [ ]:
# object detection imports
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import cv2

In [ ]:
speech = pyttsx3.init()

In [ ]:
r = sr.Recognizer()
print("What do you want me to do ?")
speech.stop()
speech.say("What do you want me to do ?")
speech.runAndWait()
with sr.Microphone() as source:
    audio = r.listen(source)

In [ ]:
'''
try:
    command =  r.recognize_sphinx(audio)
    print(command)
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print(e)
'''

In [ ]:
try:
    command =  r.recognize_google(audio)
    print(command)
except sr.UnknownValueError:
    print("Google could not understand audio")
except sr.RequestError as e:
    print(e)

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
try:
    words = word_tokenize(command)
except NameError:
    speech.stop()
    speech.say('Sorry, I didn\'t get that')
    speech.runAndWait()
words

In [ ]:
from nltk.corpus import stopwords
import string
stopwords = set(stopwords.words('english'))
wordscleaned = []
for w in words:
    if w not in stopwords and w not in string.punctuation:
        wordscleaned.append(w)
words = wordscleaned
words

In [ ]:
if nltk.pos_tag([words[0]])[0][1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD']:
    question = True
else:
    question = False
print("question = " + str(question))

In [ ]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
wordsStemmed = []
for w in words:
    wordsStemmed.append(wnl.lemmatize(w) if wnl.lemmatize(w).endswith('e') else ps.stem(w))
words = wordsStemmed
words

In [ ]:
# synonyms of find, get, fetch
find = []
findsyn = wn.synsets('find')
getsyn = wn.synsets('get')
fetchsyn = wn.synsets('fetch')
for x in findsyn + getsyn + fetchsyn:
    find.append(x.name()[:-5])
find = list(set(find))

In [ ]:
if any(x in words for x in find):
    find = True
else:
    find = False
print("find = " + str(find) )

In [ ]:
if find == True:
    speech.stop()
    speech.say("For sure !")
    speech.runAndWait()
else:
    exit()

In [ ]:
words = nltk.pos_tag(words)
words

In [ ]:
nouns = []
verbs = []
pronouns = []
adjectives = []

In [ ]:
for i,w in enumerate(words):
    if w[0] == 'bottle':
        words[i-1] = ('bottle', 'NN')
        nouns.append('bottle')
    if w[1] in ['NN', 'NNS']:
        nouns.append(w[0])
nouns

In [ ]:
nounsconcat = []
for x in nouns:
    for y in nouns:
        nounsconcat.append(x+y)
nounsconcat = list(set(nounsconcat))
nouns = nouns + nounsconcat
nouns

In [ ]:
import pickle
labelsdict = pickle.load(open('objectlabels.p', 'rb'))
labels = []
for x in labelsdict:
    labels.append(labelsdict[x])
classesdict = {}
for x in labelsdict:
    classesdict[labelsdict[x]] = x

In [ ]:
nouns2 = []
for n in nouns:
    synset = wn.synsets(n)
    for x in synset:
        nouns2.append(x.name()[:-5])
nouns.extend(nouns2)
nouns = list(set(nouns))
nouns

In [ ]:
labelsnospace = []
for x in labels:
    labelsnospace.append(x.replace(" ", ""))

In [ ]:
searchlabels = []
for i,x in enumerate(labelsnospace):
    if x in nouns:
        searchlabels.append(labels[i])
print(searchlabels)
searchclasses = []
for x in searchlabels:
    searchclasses.append(classesdict[x])
print(searchclasses)

In [ ]:
if len(searchclasses) == 0:
    speech.stop()
    speech.say("Sorry, I can't find that.")
    speech.runAndWait()
    exit()

Object detection

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../object_detection')

In [ ]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile('../object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb', 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap('../object_detection/data/mscoco_label_map.pbtxt')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    while True:
        image_valid ,image_np = cap.read()
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
    
        #selecting classes to show
        newclasses = []
        newboxes = []
        newscores = []
        num = 0
        for i,x in enumerate(classes[0]):
            if int(x) in searchclasses:
                newclasses.append(x)
                newboxes.append(boxes[0][i])
                newscores.append(scores[0][i])
                num+=1
        scores = np.array([newscores], dtype = float)
        classes = np.array([newclasses], dtype = float)
        boxes = np.array([newboxes], dtype = float)
        num = np.array([float(num)], dtype = float)
        
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
            
        cv2.imshow('image', image_np)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
a = np.array([[1.1,2.1,3.1]])
a[0][1][2]